In [1]:
import numpy as np
import scipy
import pandas as pd
from matplotlib import pyplot as plt
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('png', 'pdf')
import seaborn
import joblib
from sklearn.preprocessing import Normalizer, MaxAbsScaler

%matplotlib inline
%load_ext autoreload
%autoreload 2

from data_mining import *
from stacking.stacking import Stacking

In [2]:
residues_normalizer = Normalizer()
hbonds_normalizer = MaxAbsScaler()
solvation_normalizer = MaxAbsScaler()
backboneatom_normalizer = Normalizer()

CASP = [
    ('*residues-d4-b10-a12-c5-n0--skip_errors.mat',
     '8269eab4b9ebaa69aa1467308c48ed4f',
     lambda X: residues_normalizer.fit_transform(X),
     lambda X: residues_normalizer.transform(X)),
    ('*hbonds-b6-a6-c6-n2--skip_errors.mat',
     '8269eab4b9ebaa69aa1467308c48ed4f',
     lambda X: hbonds_normalizer.fit_transform(X),
     lambda X: hbonds_normalizer.transform(X)),
    ('*solvation-b3-a2-c15--skip_errors.mat',
     '8269eab4b9ebaa69aa1467308c48ed4f',
     lambda X: solvation_normalizer.fit_transform(X),
     lambda X: solvation_normalizer.transform(X)),
    ('*backboneatom-b25-c7-n0--residue_type_dependent--skip_errors.mat',
     'd3efaf4824785305b93d775d0e3b0a46',
     lambda X: backboneatom_normalizer.fit_transform(X),
     lambda X: backboneatom_normalizer.transform(X)),
]

NMA = [
    ('*residues-d4-b10-a12-c5-n0--skip_errors.mat',
     'c4f25d107683b57acbc6306d9f5108f2',
     lambda X: residues_normalizer.fit_transform(X),
     lambda X: residues_normalizer.transform(X)),
    ('*hbonds-b6-a6-c6-n2--skip_errors.mat',
     'c4f25d107683b57acbc6306d9f5108f2',
     lambda X: hbonds_normalizer.fit_transform(X),
     lambda X: hbonds_normalizer.transform(X)),
    ('*solvation-b3-a2-c15--skip_errors.mat',
     'c4f25d107683b57acbc6306d9f5108f2',
     lambda X: solvation_normalizer.fit_transform(X),
     lambda X: solvation_normalizer.transform(X)),
    ('*backboneatom-b25-c7-n0--residue_type_dependent--skip_errors.mat',
     'c4f25d107683b57acbc6306d9f5108f2',
     lambda X: backboneatom_normalizer.fit_transform(X),
     lambda X: backboneatom_normalizer.transform(X)),
]

In [ ]:
%%time

X, scores = combine_datasets(
    get_dataset(
        [(pattern, checksum, fit_transform)
             for (pattern, checksum, fit_transform, transform) in CASP],
        '^.*CASP([5-9]|10)/T..../.*$'
    ),
    get_dataset(
        [(pattern, checksum, transform)
             for (pattern, checksum, fit_transform, transform) in NMA],
        '^.*CASP([5-9]|10)/T..../.*$'
    )
)

Shape of the first dataset:  (160586, 4371840)
Shape of the second dataset: (160586, 216)
Shape of the final dataset:  (160586, 4372056)
Shape of the first dataset:  (160586, 4372056)
Shape of the second dataset: (160586, 138)
Shape of the final dataset:  (160586, 4372194)
Shape of the first dataset:  (160586, 4372194)
Shape of the second dataset: (160579, 239775)
Shape of the final dataset:  (160579, 4611969)
Shape of the first dataset:  (181391, 4371840)
Shape of the second dataset: (181391, 216)
Shape of the final dataset:  (181391, 4372056)
Shape of the first dataset:  (181391, 4372056)
Shape of the second dataset: (181391, 138)
Shape of the final dataset:  (181391, 4372194)
Shape of the first dataset:  (181391, 4372194)
Shape of the second dataset: (181391, 239775)
CPU times: user 24min 15s, sys: 11min 39s, total: 35min 55s
Wall time: 36min 4s


In [17]:
def dump_model(model, filename):
    joblib.dump(
        Pipeline([('scaler', CombinedScaler([residues_normalizer, hbonds_normalizer,
                                             solvation_normalizer, backboneatom_normalizer])),
                  ('scorer', model)]),
        filename, protocol=2
    )

In [5]:
X_total = X
scores_total = scores

In [6]:
np.random.seed(1)
targets = list(set([x.split('/')[6] for x in scores_total.index if x.split('/')[5] != 'CASP11']))

for p in np.linspace(0.01, 0.1, 6, endpoint=False):
    train_targets = np.random.choice(targets, int(len(targets) * p), replace=False)
    idx = np.array([x.split('/')[6] in train_targets for x in scores_total.index], dtype=bool)

    X, scores = X_total[idx], scores_total[idx]

    training_info = '_'.join(str(x) for x in X.shape)

    ridge_pipeline = Pipeline([
        ('imputer', OneHotImputer(100)),
        ('model', RRModel(normalize=False, fit_intercept=False, solver='sparse_cg', alpha=5))
    ])

    ridge_pipeline.fit(X, scores)

    dump_model(ridge_pipeline, 'ridge_pipeline_CASP_{}__{}.pkl'.format(len(set(train_targets)), training_info))

# Regression

In [ ]:
training_info = 'GDT_TS_model_' + '_'.join(str(x) for x in X.shape)

## Ridge Regression

In [ ]:
%%time

ridge_pipeline = RRModel(normalize=False, fit_intercept=False, solver='sparse_cg', alpha=5)

ridge_pipeline.fit(X, scores)

In [ ]:
dump_model(ridge_pipeline, 'ridge_pipeline_{}.pkl'.format(training_info))

## Weighted Ridge Regression

In [ ]:
%%time

ridge_pipeline = RRModel(normalize=False, fit_intercept=False, solver='sparse_cg', alpha=5)

ridge_pipeline.fit(X, scores, model__sample_weight=scores['GDT-TS-score'].values ** 2)

In [ ]:
dump_model(ridge_pipeline, 'weighted_ridge_pipeline_{}.pkl'.format(training_info))

# Ranking

In [18]:
training_info = '_'.join(str(x) for x in X.shape)

## Ridge Regression

In [ ]:
%%time

ridge_pipeline = Pipeline([
    ('imputer', OneHotImputer(100)),
    ('model', RRModel(normalize=False, fit_intercept=False, solver='sparse_cg', alpha=5))
])

ridge_pipeline.fit(X, scores)

In [ ]:
dump_model(ridge_pipeline, 'ridge_pipeline_{}.pkl'.format(training_info))

In [ ]:
%%time

ridge_pipeline = Pipeline([
    ('imputer', OneHotImputer(100)),
    ('model', RRModel(normalize=False, fit_intercept=False, solver='sparse_cg', alpha=5))
])

ridge_pipeline.fit(X[scores.RMSD.values < 15], scores[scores.RMSD.values < 15])

In [ ]:
dump_model(ridge_pipeline, 'ridge_pipeline_{}.pkl'.format(training_info + '_purified_15A'))

## Weighted Ridge Regression

In [ ]:
%%time

ridge_pipeline = Pipeline([
    ('imputer', OneHotImputer(100)),
    ('model', RRModel(normalize=False, fit_intercept=False, solver='sparse_cg', alpha=5))
])

ridge_pipeline.fit(X, scores, model__sample_weight=scores['GDT-TS-score'].values ** 2)

In [ ]:
dump_model(ridge_pipeline, 'weighted_ridge_pipeline_{}.pkl'.format(training_info))

## Ridge Native

In [ ]:
%%time

ridge_native = RRModelNative(normalize=False, fit_intercept=False, solver='sparse_cg', alpha=0.8)

ridge_native.fit(X, scores)

In [ ]:
dump_model(ridge_native, 'ridge_native_{}.pkl'.format(training_info))

## Ridge(LR)

In [ ]:
%%time

ridge__LR = Stacking(
    base_estimators=[
        (Pipeline([('imputer', OneHotImputer(5)),
                   ('model', LRModel(fit_intercept=True, penalty='l2', solver='lbfgs',
                                     C=100, class_weight={False: 1, True: 20}))]).fit, base_predict)
    ],
    meta_fitter=Pipeline([('imputer', OneHotImputer(100)),
                          ('model', RRModel(normalize=False, fit_intercept=False,
                                            solver='sparse_cg', alpha=5))]).fit,
    get_folds=get_folds,
    n_folds=8,
    extend_meta=True
)

ridge__LR.fit(X, scores)

In [ ]:
dump_model(ridge__LR, 'ridge__LR_{}.pkl'.format(training_info))

## Ridge(RidgeNative)

In [ ]:
%%time

ridge__ridge_native = Stacking(
    base_estimators=[
        (RRModelNative(normalize=False, fit_intercept=False, solver='sparse_cg', alpha=0.8).fit, base_predict)
    ],
    meta_fitter=Pipeline([('imputer', OneHotImputer(100)),
                          ('model', RRModel(normalize=False, fit_intercept=False,
                                            solver='sparse_cg', alpha=5))]).fit,
    get_folds=get_folds,
    n_folds=8,
    extend_meta=True
)

ridge__ridge_native.fit(X, scores)

In [ ]:
dump_model(ridge__ridge_native, 'ridge__ridge_native_{}.pkl'.format(training_info))

## Ridge(RidgeNative, Ridge, LR)

In [ ]:
%%time

ridge_all = Stacking(
    base_estimators=[
        (Pipeline([('imputer', OneHotImputer(5)),
                   ('model', LRModel(fit_intercept=True, penalty='l2', solver='lbfgs',
                                     C=100, class_weight={False: 1, True: 20}))]).fit, base_predict),
        (RRModelNative(normalize=False, fit_intercept=False, solver='sparse_cg', alpha=0.8).fit, base_predict),
        (RRModelNative(normalize=False, fit_intercept=True, alpha=8).fit, base_predict),
        (Pipeline([('imputer', OneHotImputer(100)),
                          ('model', RRModel(normalize=False, fit_intercept=False,
                                            solver='sparse_cg', alpha=5))]).fit, base_predict),
    ],
    meta_fitter=Pipeline([('imputer', OneHotImputer(1)),
                          ('model', RRModel(normalize=False, fit_intercept=True, alpha=0.01))]).fit,
    get_folds=get_folds,
    n_folds=8,
    extend_meta=False
)

ridge_all.fit(X, scores)

In [ ]:
dump_model(ridge_all, 'ridge_all_{}.pkl'.format(training_info))

## Ranking Stacking

In [ ]:
%%time

ranking_stacking = Stacking(
    base_estimators=[
        (Pipeline([('imputer', OneHotImputer(5)),
                   ('model', LRModel(fit_intercept=True, penalty='l2', solver='lbfgs',
                                     C=100, class_weight={False: 1, True: 20}))]).fit, base_predict),
        (Pipeline([('imputer', OneHotImputer(100)),
                   ('model', RRModel(normalize=False, fit_intercept=False,
                                     solver='sparse_cg', alpha=5))]).fit, base_predict),
        (RRModelNative(normalize=False, fit_intercept=False, solver='sparse_cg', alpha=1).fit, base_predict),
    ],
    meta_fitter=RankingLR(penalty='l1', fit_intercept=False, C=1, rank_transform_threshold=0.01).fit,
    get_folds=get_folds,
    n_folds=8,
    extend_meta=False
)

ranking_stacking.fit(X, scores)

In [ ]:
dump_model(ranking_stacking, 'ranking_stacking_{}.pkl'.format(training_info))